## Importando os dados que iremos utilizar


In [1]:
import pandas as pd


dados = pd.read_pickle("dados\dados_tratados")
dados = dados.drop(
    columns=["infection.county", "occupation", "data", "detection.numerico"]
)  # deletando dados que não vão ser utilizados
dados = dados.reindex()  # apenas para ter certezar que os index estão corretos